# Sentiment Analysis by hand and with NLTK 

# Setup

## Install dependencies

In [258]:
import sys
!{sys.executable} -m pip install nltk
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install sklearn

smart-open 1.5.6 requires boto3, which is not installed.
qiskit 0.4.5 has requirement matplotlib<=2.1,>=2.0, but you'll have matplotlib 2.1.2 which is incompatible.
qiskit 0.4.5 has requirement networkx<1.12,>=1.11, but you'll have networkx 2.0 which is incompatible.
qiskit 0.4.5 has requirement numpy<=1.14,>=1.13, but you'll have numpy 1.14.3 which is incompatible.
networkx 2.0 has requirement decorator>=4.1.0, but you'll have decorator 4.0.11 which is incompatible.
awsebcli 3.10.1 has requirement requests<=2.9.1,>=2.6.1, but you'll have requests 2.18.4 which is incompatible.


smart-open 1.5.6 requires boto3, which is not installed.
qiskit 0.4.5 has requirement matplotlib<=2.1,>=2.0, but you'll have matplotlib 2.1.2 which is incompatible.
qiskit 0.4.5 has requirement networkx<1.12,>=1.11, but you'll have networkx 2.0 which is incompatible.
qiskit 0.4.5 has requirement numpy<=1.14,>=1.13, but you'll have numpy 1.14.3 which is incompatible.
networkx 2.0 has requirement decorator>=4.1.0, but you'll have decorator 4.0.11 which is incompatible.
awsebcli 3.10.1 has requirement requests<=2.9.1,>=2.6.1, but you'll have requests 2.18.4 which is incompatible.


  Running setup.py bdist_wheel for sklearn: started
  Running setup.py bdist_wheel for sklearn: finished with status 'done'
  Stored in directory: C:\Users\Tiger\AppData\Local\pip\Cache\wheels\76\03\bb\589d421d27431bcd2c6da284d5f2286c8e3b2ea3cf1594c074
Successfully built sklearn


smart-open 1.5.6 requires boto3, which is not installed.
qiskit 0.4.5 has requirement matplotlib<=2.1,>=2.0, but you'll have matplotlib 2.1.2 which is incompatible.
qiskit 0.4.5 has requirement networkx<1.12,>=1.11, but you'll have networkx 2.0 which is incompatible.
qiskit 0.4.5 has requirement numpy<=1.14,>=1.13, but you'll have numpy 1.14.3 which is incompatible.
networkx 2.0 has requirement decorator>=4.1.0, but you'll have decorator 4.0.11 which is incompatible.
awsebcli 3.10.1 has requirement requests<=2.9.1,>=2.6.1, but you'll have requests 2.18.4 which is incompatible.


## Import dependencies

In [259]:
import nltk
import csv
import pandas as pd

#gives us the list of punctuations
import string

# for stopword removal. stopwords are like "the", "and", "over"
from nltk.corpus import stopwords
nltk.download('stopwords')

# gives us functionality to stem a world. e.g. running -> run
from nltk.stem import PorterStemmer

# gives us functionality of a blackbox sentiment analysis analyser (classes: positive, negative, neutral)
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon') #pre-trained lexicon similar to AFINN

# 
from nltk.tokenize import word_tokenize
nltk.download('punkt')

# calculating metrics of AFINN
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Tiger\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Tiger\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Tiger\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [217]:
# Filepath to dataset
fpDataset = './data/customer-feedback_full_cleaned_1000.xlsx'

#Load Excel file into a DataFrame
dfExcelWorkbook = pd.read_excel(fpDataset, sheet_name=None)
sheets = list(dfExcelWorkbook.keys())
dfData = dfExcelWorkbook[sheets[0]]
dfData_backup = dfData.copy()

In [218]:
dfData

,FEEDBACK,RATING
0,never got clean glasses in Warsaw either.,0
1,The bed in the Radisson Bleu was not comfortab...,1
2,Michael was an excellent tour director. He wen...,1
3,Krakow Hotel was below my expectations because...,0
4,All the city tour guides have been excellent a...,1
5,The bed in the Radisson Bleu was not comfortab...,0
6,The Prague hotel should provide in-room intern...,0
7,Michael (Tour Director) was brilliant! Thomas ...,1
8,The entire voyage was very well done by Viking...,0
9,Michael was excellent. The Prague hotel should...,1


In [219]:
# first feedback
dfData['FEEDBACK'].loc[0]

'never got clean glasses in Warsaw either.'

In [220]:
# filter for only positive ratings
dfData[dfData['RATING'] == 1]

,FEEDBACK,RATING
1,The bed in the Radisson Bleu was not comfortab...,1
2,Michael was an excellent tour director. He wen...,1
4,All the city tour guides have been excellent a...,1
7,Michael (Tour Director) was brilliant! Thomas ...,1
9,Michael was excellent. The Prague hotel should...,1
10,Our driver was very good. Local tour guide all...,1
11,"Vikig personnel were ""far above expectations"" ...",1
12,Michael was excellent.,1
13,"Michael, our tour director was outstanding. Hi...",1
14,A great trip,1


In [221]:
#loading the AFINN mapping
lol = list(csv.reader(open('data/AFINN-111.txt', 'r'), delimiter='\t')) #load afinn into list of lists
afinn = {d[0]: int(d[1]) for d in lol} #create afinn dictionary

def afinnScore(word):
    return afinn[word.lower()] if word.lower() in afinn else 0

In [222]:
afinn

{'abandon': -2,
 'abandoned': -2,
 'abandons': -2,
 'abducted': -2,
 'abduction': -2,
 'abductions': -2,
 'abhor': -3,
 'abhorred': -3,
 'abhorrent': -3,
 'abhors': -3,
 'abilities': 2,
 'ability': 2,
 'aboard': 1,
 'absentee': -1,
 'absentees': -1,
 'absolve': 2,
 'absolved': 2,
 'absolves': 2,
 'absolving': 2,
 'absorbed': 1,
 'abuse': -3,
 'abused': -3,
 'abuses': -3,
 'abusive': -3,
 'accept': 1,
 'accepted': 1,
 'accepting': 1,
 'accepts': 1,
 'accident': -2,
 'accidental': -2,
 'accidentally': -2,
 'accidents': -2,
 'accomplish': 2,
 'accomplished': 2,
 'accomplishes': 2,
 'accusation': -2,
 'accusations': -2,
 'accuse': -2,
 'accused': -2,
 'accuses': -2,
 'accusing': -2,
 'ache': -2,
 'achievable': 1,
 'aching': -2,
 'acquit': 2,
 'acquits': 2,
 'acquitted': 2,
 'acquitting': 2,
 'acrimonious': -3,
 'active': 1,
 'adequate': 1,
 'admire': 3,
 'admired': 3,
 'admires': 3,
 'admiring': 3,
 'admit': -1,
 'admits': -1,
 'admitted': -1,
 'admonish': -2,
 'admonished': -2,
 'adopt': 

## Getting the sentiment score for the Feedbacks

In [223]:
#get the afinn scores for one example
sampleSentence = dfData['FEEDBACK'].loc[990]

wordList = sampleSentence.split(' ')
wordList_scores = [afinnScore(word) for word in wordList]

print(sampleSentence)
print(wordList_scores)

Everyone representing Viking was outstanding! A perfect crew!
[0, 0, 0, 0, 0, 0, 3, 0]


In [224]:
#which words got scored?

#get all scores in a dictionary
scoredWords = dict(zip(wordList,wordList_scores))
#get only the ones with value != 0
scoredWords = {key: val for key, val in scoredWords.items() if val != 0}
print(scoredWords)

{'perfect': 3}


In [225]:
#The word OUTSTANDING is also in AFINN. but it has some punctuation on it. let's remove the punctuation

## Remove punctuation 

In [226]:
def removePunctuation(sentence):
    exclude = set(string.punctuation)
    return ''.join(ch for ch in sentence if ch not in exclude)

sentences = [removePunctuation(sentence) for sentence in sentences]
print(sampleSentence)

Everyone representing Viking was outstanding! A perfect crew!


In [227]:
# lets apply to the whole dataset
dfData['FEEDBACK'] = dfData['FEEDBACK'].apply(removePunctuation)
sampleSentence = dfData['FEEDBACK'].loc[990]

In [228]:
#let's put the afinn score for a sentence into a function and then apply to our sample sentence again 

def getAfinnScores(sentence): 
    wordList = sentence.split(' ')
    wordList_scores = [afinnScore(word) for word in wordList] #repeating words are respected
    sentenceScore = sum(wordList_scores)
    
    scoredWords = dict(zip(wordList,wordList_scores))
    scoredWords = {key: val for key, val in scoredWords.items() if val != 0} #only get the scored words that matter
    return sentenceScore,scoredWords

In [229]:
getAfinnScores(sampleSentence)

(8, {'outstanding': 5, 'perfect': 3})

In [230]:
#Now let's apply this function to the whole dataset and add the information to the frame

In [231]:
dfData['AFINN-score'] = dfData['FEEDBACK'].apply(getAfinnScores)

In [232]:
dfData

,FEEDBACK,RATING,AFINN-score
0,never got clean glasses in Warsaw either,0,"(2, {'clean': 2})"
1,The bed in the Radisson Bleu was not comfortab...,1,"(15, {'comfortable': 2, 'excellent': 3, 'good'..."
2,Michael was an excellent tour director He went...,1,"(9, {'excellent': 3, 'comfortable': 2, 'safe':..."
3,Krakow Hotel was below my expectations because...,0,"(8, {'hard': -1, 'top': 2, 'warm': 1, 'cool': ..."
4,All the city tour guides have been excellent a...,1,"(13, {'excellent': 3, 'good': 3, 'care': 2, 's..."
5,The bed in the Radisson Bleu was not comfortab...,0,"(15, {'comfortable': 2, 'excellent': 3, 'good'..."
6,The Prague hotel should provide inroom interne...,0,"(3, {'tired': -2, 'outstanding': 5})"
7,Michael Tour Director was brilliant Thomas Bus...,1,"(10, {'brilliant': 4, 'excellent': 3, 'safe': ..."
8,The entire voyage was very well done by Viking...,0,"(-1, {'no': -1})"
9,Michael was excellent The Prague hotel should ...,1,"(6, {'excellent': 3, 'tired': -2, 'outstanding..."


## Removal of Stopwords using NLTK

In [233]:
# now we remove stopwords that are not required in the dataset, just to clean it up
def removeStopWords(sentence):
    stopwordList = stopwords.words("english")
    wordList = [word for word in sentence.split(' ') if removePunctuation(word.lower()) not in stopwordList]
    return ' '.join(wordList)

#see example:
print(removePunctuation(sampleSentence), end='\n-----------\n')
print(removeStopWords(removePunctuation(sampleSentence)))

Everyone representing Viking was outstanding A perfect crew
-----------
Everyone representing Viking outstanding perfect crew


In [234]:
# apply it to the whole dataset
dfData['FEEDBACK'] = dfData['FEEDBACK'].apply(removeStopWords)
sampleSentence = dfData['FEEDBACK'].loc[990]

In [254]:
def normalize_afinn(affinScore):
    if affinScore > 0: return 1
    else: return 0
    

dfScore = pd.DataFrame(list(dfData['AFINN-score']),columns=['sentence_score', 'word_scores'])
dfData['AFINN-score-normalized'] = dfScore['sentence_score'].apply(normalize_afinn)

In [255]:
dfData

,FEEDBACK,RATING,AFINN-score,AFINN-score-normalized
0,never got clean glasses Warsaw either,0,"(2, {'clean': 2})",1
1,bed Radisson Bleu comfortable However breakfas...,1,"(15, {'comfortable': 2, 'excellent': 3, 'good'...",1
2,Michael excellent tour director went way make ...,1,"(9, {'excellent': 3, 'comfortable': 2, 'safe':...",1
3,Krakow Hotel expectations beds way hard Michae...,0,"(8, {'hard': -1, 'top': 2, 'warm': 1, 'cool': ...",1
4,city tour guides excellent nd personable well ...,1,"(13, {'excellent': 3, 'good': 3, 'care': 2, 's...",1
5,bed Radisson Bleu comfortable However breakfas...,0,"(15, {'comfortable': 2, 'excellent': 3, 'good'...",1
6,Prague hotel provide inroom internet krakow ho...,0,"(3, {'tired': -2, 'outstanding': 5})",1
7,Michael Tour Director brilliant Thomas Bus dri...,1,"(10, {'brilliant': 4, 'excellent': 3, 'safe': ...",1
8,entire voyage well done Viking Luggage transfe...,0,"(-1, {'no': -1})",0
9,Michael excellent Prague hotel provide inroom ...,1,"(6, {'excellent': 3, 'tired': -2, 'outstanding...",1


In [260]:
y_test = dfData['RATING']
y_pred = dfData['AFINN-score-normalized']
print(f1_score(y_test, y_pred, average="macro"))
print(precision_score(y_test, y_pred, average="macro"))
print(recall_score(y_test, y_pred, average="macro"))

0.7145833333333332
0.7381325336153767
0.7147485300568659


## Word stemming using NLTK's PorterStemmer

In [235]:
#Word stemming means trimming the word to its lexicographical stem.
# this way multiple derivatives of words can be categorized as one. e.g. running -> run

# In general it is good practice to stem all the words,
# as some algorithms assume that the words have been stemmed with a specific algorithm

def stemWords(sentence):
    wordList = sentence.split(' ')
    ps = PorterStemmer()
    return ' '.join([ps.stem(word) for word in wordList])

In [236]:
# this stemmer is highly rule-based rather than lexicograph. It follows the principle of:
# "the purpose of stemming is to bring variant forms of a word together, not to map a word onto its ‘paradigm’ form."

print(sampleSentence)
print(stemWords(sampleSentence))

Everyone representing Viking outstanding perfect crew
everyon repres vike outstand perfect crew


In [237]:
#let's NOT apply to the whole dataset just yet

#dfData['FEEDBACK'] = dfData['FEEDBACK'].apply(stemWords)
#sampleSentence = dfData['FEEDBACK'].loc[990]

In [238]:
#one word of caution: there are several algorithms that assume different initial conditions.
# an example is here: the goal of stemming was to make words more generalizeable. but the AFINN mapping assumes no stemming

# for example for words in the "family" of "affect", AFINN has scores for the following:
afinn_affect = [word for word in afinn if (word.startswith('affect'))]
print([getAfinnScores(word) for word in afinn_affect], end ="\n----------\n")

# the stemmed versions are as follows:
ps_affect = [PorterStemmer().stem(word) for word in afinn_affect]
print(ps_affect, end ="\n----------\n")

#and these are the scores after the stemming
print([getAfinnScores(word) for word in ps_affect], end ="\n----------\n")

[(-1, {'affected': -1}), (3, {'affection': 3}), (3, {'affectionate': 3})]
----------
['affect', 'affect', 'affection']
----------
[(0, {}), (0, {}), (3, {'affection': 3})]
----------


In [239]:
#the afinn score after stemming is negatively affected
print(getAfinnScores(stemWords(sampleSentence)), end ="\n-------\n")

# lThis is because the stemmer changed the word OUTSTAND to something that is not mapped in AFINN
print([PorterStemmer().stem(word) for word in sampleSentence.split(' ')]) 

(3, {'perfect': 3})
-------
['everyon', 'repres', 'vike', 'outstand', 'perfect', 'crew']


## Getting scores from pre-trained black box model with NLTK 

In [240]:
sid = SentimentIntensityAnalyzer()

# prints the scores for some feedbacks
for sentence in dfData['FEEDBACK'].loc[4:5]:
    print(sid.polarity_scores(sentence))
    print(sentence, end='\n------------\n')
    

# here the vader model is already a trained model. 
# we use the model to calculate the sentiment score for the sentences in our dataset

#source: https://opensourceforu.com/2016/12/analysing-sentiments-nltk/

{'neg': 0.0, 'neu': 0.449, 'pos': 0.551, 'compound': 0.9451}
city tour guides excellent nd personable well knowledgable Michael excellent tour director took good care us entire trip 5 solid stars
------------
{'neg': 0.0, 'neu': 0.366, 'pos': 0.634, 'compound': 0.9538}
bed Radisson Bleu comfortable However breakfast excellent good food excellent service wonderful time travel Viking
------------


In [213]:
# as we do not know on which basis this blackbox algorithm works, 
# one has to test whether stemming, removing punctuations etc gives a better or worse result

#here we use the unmodified dfData_backup
for sentence in dfData_backup['FEEDBACK'].loc[4:5]:
    print(sid.polarity_scores(sentence))
    print(sentence, end='\n------------\n')

{'neg': 0.0, 'neu': 0.625, 'pos': 0.375, 'compound': 0.9519}
All the city tour guides have been excellent a nd very personable as well as very knowledgable. Michael was an excellent tour director and took very good care of us the entire trip. 5 solid stars for him
------------
{'neg': 0.065, 'neu': 0.599, 'pos': 0.336, 'compound': 0.9062}
The bed in the Radisson Bleu was not comfortable. However, the breakfast was excellent with good food and excellent service. WE had a wonderful time and will travel with Viking again
------------


## Training own generic classifier 

In [264]:
#now we use the labels from the dataset

# here we do not have a pre-trained model. we use our own model to train a generic classifier
    
# Step 1 – Training data
#labels = ['neg','pos','neg','pos','pos']
dataset = list(zip(dfData["FEEDBACK"],dfData["RATING"]))
  
# Step 2
dictionary = set(word.lower() for passage in dataset for word in word_tokenize(passage[0]))
  
# Step 3
t = [({word: (word in word_tokenize(x[0])) for word in dictionary}, x[1]) for x in dataset]
  
# Step 4 – the classifier is trained with sample data
classifier = nltk.NaiveBayesClassifier.train(t)

In [267]:
classifier.show_most_informative_features()

Most Informative Features
                  driver = True                1 : 0      =     15.7 : 1.0
                    long = True                0 : 1      =     13.8 : 1.0
             informative = True                1 : 0      =     11.1 : 1.0
           knowledgeable = True                1 : 0      =     11.1 : 1.0
                    crew = True                1 : 0      =     10.6 : 1.0
                       1 = True                0 : 1      =     10.5 : 1.0
                    poor = True                0 : 1      =     10.5 : 1.0
                   night = True                0 : 1      =     10.5 : 1.0
                    slow = True                0 : 1      =      9.7 : 1.0
                friendly = True                1 : 0      =      9.4 : 1.0


In [269]:
test_data = sampleSentence
print(sampleSentence)

test_data_features = {word.lower(): (word in word_tokenize(test_data.lower())) for word in dictionary}

distribution = classifier.prob_classify(test_data_features)
for label in distribution.samples():
    print("%s: %f" % (label, distribution.prob(label)))

Everyone representing Viking outstanding perfect crew
0: 0.000017
1: 0.999983


In [270]:
test_data_features

{'charming': False,
 'otherwise': False,
 'course': False,
 'alltogether': False,
 'member': False,
 'excellents': False,
 'depending': False,
 'focus': False,
 'disney': False,
 'leak': False,
 'venue': False,
 'pastry': False,
 'claps': False,
 'booked': False,
 'organizer': False,
 'mattress': False,
 'les': False,
 'river': False,
 'gret': False,
 'space': False,
 'look': False,
 'vivier': False,
 'songs': False,
 'litle': False,
 'lacking': False,
 'exceptional': False,
 'staff': False,
 'due': False,
 'self': False,
 'wiaiting': False,
 'uneven': False,
 'showers': False,
 'tiem': False,
 'refreshing': False,
 'lack': False,
 'giving': False,
 'watching': False,
 'bathroom': False,
 'belive': False,
 'absolutly': False,
 'cumbersome': False,
 'winds': False,
 'paced': False,
 'berlin': False,
 'assisting': False,
 'terrific': False,
 'adapt': False,
 'business': False,
 'excursions': False,
 'often': False,
 'pature': False,
 'veronika': False,
 'rst': False,
 'fulfilled': False,